In [4]:
import boto3

In [2]:
s3 = boto3.client('s3')

In [3]:
s3.create_bucket(Bucket='test-bucket-yingzi')

{'ResponseMetadata': {'RequestId': 'TQK6RH09SNTN6E2W',
  'HostId': 'nqwsDOFpEtuu5hVMSD5eXCcriJY2v174OvNZSFXkHtETVK5snf4wHnLQq/W5qBl6e8kd3pdnzz0=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'nqwsDOFpEtuu5hVMSD5eXCcriJY2v174OvNZSFXkHtETVK5snf4wHnLQq/W5qBl6e8kd3pdnzz0=',
   'x-amz-request-id': 'TQK6RH09SNTN6E2W',
   'date': 'Thu, 25 May 2023 04:40:53 GMT',
   'location': '/test-bucket-yingzi',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'Location': '/test-bucket-yingzi'}

Create EMR Cluster

In [68]:
"""
aws emr create-cluster \
    --name "Spark Cluster" \
    --release-label "emr-6.2.0" \
    --applications Name=Hadoop Name=Hive Name=JupyterEnterpriseGateway Name=JupyterHub Name=Livy Name=Pig Name=Spark Name=Tez \
    --instance-type m5.xlarge \
    --instance-count 5 \
    --use-default-roles \
    --region us-east-1 \
    --ec2-attributes '{"KeyName": "vockey"}' \
    --configurations '[{"Classification": "jupyter-s3-conf", "Properties": {"s3.persistence.enabled": "true", "s3.persistence.bucket": "test-bucket-yingzi"}}]'
"""    

'\naws emr create-cluster     --name "Spark Cluster"     --release-label "emr-6.2.0"     --applications Name=Hadoop Name=Hive Name=JupyterEnterpriseGateway Name=JupyterHub Name=Livy Name=Pig Name=Spark Name=Tez     --instance-type m5.xlarge     --instance-count 5     --use-default-roles     --region us-east-1     --ec2-attributes \'{"KeyName": "vockey"}\'     --configurations \'[{"Classification": "jupyter-s3-conf", "Properties": {"s3.persistence.enabled": "true", "s3.persistence.bucket": "test-bucket-yingzi"}}]\'\n'

ssh -i /Users/axyz1220/Downloads/macs30123/A3/labsuser.pem -NL 9443:localhost:9443 hadoop@ec2-54-224-199-218.compute-1.amazonaws.com

https://localhost:9443

Export RDS snapshot and Store the data in parquet format

In [1]:
import boto3
import json

In [2]:
rds = boto3.client('rds')

In [67]:
# create db instance
response = rds.create_db_instance(
    DBInstanceIdentifier='relational-db2',
    DBName='reddit_scrapes',
    MasterUsername='username',
    MasterUserPassword='password',
    DBInstanceClass='db.t2.micro',
    Engine='mysql',
    AllocatedStorage=5
)
# Wait until DB is available to continue
rds.get_waiter('db_instance_available') \
   .wait(DBInstanceIdentifier='relational-db2')

In [49]:
# Describe where DB is available and on what port
db = rds.describe_db_instances()['DBInstances'][1]
ENDPOINT = db['Endpoint']['Address']
PORT = db['Endpoint']['Port']
DBID = db['DBInstanceIdentifier']

print(DBID,
      "is available at", ENDPOINT,
      "on Port", PORT,
     )   

relational-db2 is available at relational-db2.csndcbhvlggl.us-east-1.rds.amazonaws.com on Port 3306


In [50]:
# Get Name of Security Group
SGNAME = db['VpcSecurityGroups'][0]['VpcSecurityGroupId']

# Adjust Permissions for that security group so that we can access it on Port 3306
# If already SG is already adjusted, print this out
try:
    ec2 = boto3.client('ec2')
    data = ec2.authorize_security_group_ingress(
            GroupId=SGNAME,
            IpPermissions=[
                {'IpProtocol': 'tcp',
                 'FromPort': PORT,
                 'ToPort': PORT,
                 'IpRanges': [{'CidrIp': '0.0.0.0/0'}]}
            ]
    )
except ec2.exceptions.ClientError as e:
    if e.response["Error"]["Code"] == 'InvalidPermission.Duplicate':
        print("Permissions already adjusted.")
    else:
        print(e)

Permissions already adjusted.


In [16]:
import mysql.connector

In [55]:
conn =  mysql.connector.connect(host=ENDPOINT,
                                user="username",
                                passwd="password", 
                                port=PORT, 
                                database='reddit_scrapes')
cur = conn.cursor()

In [56]:
cur.execute("SHOW TABLES")
cur.fetchall()

[('comments',), ('posts',)]

In [57]:
cur.execute("SELECT * FROM comments")
rows = cur.fetchall()
print(len(list(rows)))
print(list(rows)[0])

20889
('Firearms', None, 33, 'The founders were men committing treason against the ruling state.\n\nWhy would they ever turn around and say only the state can dictate who has what weapons and for what purpose\n\nFn regarded af', 'https://www.reddit.com/r/Firearms/comments/13ihyp5/vox_a_new_supreme_court_case_seeks_to_legalize/', 'jk9wvum')


In [59]:
! pip install pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 7.9 MB/s eta 0:00:0000:0100:01


In [60]:
import pyarrow as pa
import pyarrow.parquet as pq

In [62]:
import pandas as pd

In [63]:
column_names = [i[0] for i in cur.description]    
df_comments = pd.DataFrame(rows, columns=column_names)   

In [64]:
table = pa.Table.from_pandas(df_comments)

# Define the output Parquet file path
output_file = "comments.parquet"

# Write the PyArrow Table to Parquet file
pq.write_table(table, output_file)

In [65]:
cur.execute("SELECT * FROM posts")
rows = cur.fetchall()

column_names = [i[0] for i in cur.description]    
df_posts = pd.DataFrame(rows, columns=column_names)  

table = pa.Table.from_pandas(df_posts)

# Define the output Parquet file path
output_file = "posts.parquet"

# Write the PyArrow Table to Parquet file
pq.write_table(table, output_file)

In [66]:
cur.close()
conn.close()

In [ ]:
def df_to_pd(df, body, valence, moral, frac):
    if body:
        col2 = 'body_valence'
    else:
        col2 = 'title_valence'
    
    if moral == 'high':
        mc = 1
    else:
        mc = 0
     
    df_select = df.filter((col('moral_conviction')==mc) & (col(col2)==valence))
    df_select = df_select.filter(col('engagement') > 0)
    df_sample = df_select.sample(fraction=frac)
    df_sample = df_sample.withColumn('log_engagement', log(col('engagement')))

    pd_sample = df_sample.toPandas()
    return pd_sample

In [ ]:
def linear_reg_plot(pd, model, body, valence, moral, content):
    coeff = model.coefficients[0]
    intercept = model.intercept
    
    if body:
        column = 'abs_body_intensity'
    else:
        column = 'abs_title_intensity'
        
    
    if valence == 'pos':
        label1 = 'Positive'
    else:
        label1 = 'Negative'
        
    if moral == 'high':
        label2 = 'High'
    else:
        label2 = 'Low'
    
    pred = pd[column] * coeff + intercept

    plt.close()
    
    plt.scatter(pd[column], pd['log_engagement'])
    plt.plot(pd[column], pred, color='red', label='Linear Regression')
    
    plt.title(f'Linear Rregression for {content} with {label2}-level Moral Conviction',
             fontsize = 10)
    plt.xlabel(f'{label1} Absolute Intensity')
    plt.ylabel('Log-transformed Engagement')
    plt.legend()
    
    return plt

In [ ]:
posts_body_neg_high_pd = df_to_pd(df_posts, True, 'neg', 'high', 0.3)

In [ ]:
plt = linear_reg_plot(posts_body_neg_high_pd, lr_body_neg_high, True, 'neg', 'high', "Posts' Body")
%matplot plt

In [ ]:
posts_body_pos_low_pd = df_to_pd(df_posts, True, 'pos', 'low', 0.3)

In [ ]:
plt = linear_reg_plot(posts_body_pos_low_pd, lr_body_pos_low, True, 'pos', 'low', "Posts' Body")
%matplot plt

In [ ]:
# remove duplicate posts

data_posts = data_posts.dropDuplicates(['body'])

In [ ]:
# remove duplicate comments

data_comments = data_comments.dropDuplicates(['comment_body'])